In [1]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time
import pickle

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import sys
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import *
from metpy.calc import lat_lon_grid_deltas
from pioneer.das.api.platform import Platform
from scipy.spatial.transform import Rotation as R
from pioneer.das.api.egomotion.imu_egomotion_provider import IMUEgomotionProvider as emp 
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

2023-05-03 11:50:27.374382: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 11:50:28.002983: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-03 11:50:29.108004: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/derm/anaconda3/envs/py39/lib/python3.9/site-packages/cv2/../../lib64:
2023-05-03 11:50:29.108113: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_p

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-05-03 11:50:31.039308: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 11:50:31.041014: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-03 11:50:31.041503: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-03 11:50:31.041787: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


# Goal: Simultaneously Estimate Translation, Rotation, and Distortion

## $\mathbf{A} = [\hat{X}_{ij}, \hat{m}_{ij}] = 
\begin{bmatrix}
% x, ~ y, ~ z, ~ \phi, ~ \theta, ~ \psi, ~ x^+, ~y^+, ~z^+, ~\phi^+, ~\theta^+, ~\psi^+ \\
x, ~ y, ~ z, ~ \phi, ~ \theta, ~ \psi, ~ x^+, ~y^+, ~z^+, ~\phi^+, ~\theta^+, ~\psi^+ \\
\end{bmatrix}$

We can use newton-raphson to find A

<!-- ## $y_i = \mathbf{h}(y_j, \hat{X}_{ij}, \hat{m}_{ij}) + \mathbf{H}_m \delta m + \mathbf{H}_x \delta x + \text{H.O.T.}$ -->

## $y_i = \mathbf{h}(y_j, \hat{A}_{ij}) + \mathbf{H}_A \delta A + \text{H.O.T.}$

## $\mathbf{H}_A \in \mathbb{R}^{4N \times 12} $



### $\mathbf{H}_A = [H_X, H_m]$

# Run Demo

In [192]:
# load point cloud
# # no distortion
# old_cloud =  np.load("sample_data/paper_figures/case1/raw_frame_0.npy")
# # m_hat = np.array([0., 0., 0., 0., 0., 0.0]) #actual motion
# m_hat = np.array([-3., 0., 0., 0., 0., 0.1]) #test wrap around

# movement in x
old_cloud =  np.load("sample_data/paper_figures/case2/raw_frame_3.npy") 
m_hat = np.array([3, 0, 0., 0., 0., 0])
# m_hat = np.array([3., 0, 0., 0., 0., -0.3]) #FOR DEBUG-- deform just a little
gt =  np.load("sample_data/paper_figures/case2/base_vel_2.npy")

# # movement in x, y, & yaw
# old_cloud =  np.load("sample_data/paper_figures/case3/raw_frame_1.npy") 
# # m_hat = np.array([3, -1, 0., 0., 0., -1])
# m_hat = np.array([3, -1, 0., 0., 0., -0.86]) #FOR DEBUG-- deform a little extra
# # m_hat = np.array([0., 0., 0., 0., 0., 0.0]) #FOR DEBUG-- no deformation
# gt =  np.load("sample_data/paper_figures/case3/base_vel_2.npy")
# # print(gt) 

# period_lidar = 1
# t_scale = (2*np.pi)/(-m_hat[-1] + (2*np.pi/period_lidar))
# print(t_scale)
# m_hat = m_hat*t_scale
# # m_hat[-1] = m_hat[-1]*t_scaled
# print(m_hat)

#downsample
old_cloud = old_cloud[::5,:]

In [ ]:
from remove_motion_basic import linear_correction_old as lc 

#apply ground truth distortion according to m_hat
new_cloud = lc(old_cloud, m_hat) 
#set ground truth transform between clouds
# X_gt = np.array([0, 0., 0.0, 0.0, 0.0, 0.0])
# X_gt = np.array([1.5, 0.5, 0.03, 0.03, 0.03, 0.25])
# X_gt = np.array([1.5, -3, 0.1, 0.2, 0.03, -0.15])
X_gt = np.array([-2.5, 0., 0.0, 0.1, 0.1, 0.1])

# add noise
old_cloud += 0.01*np.random.randn(np.shape(old_cloud)[0], 3)

# #for fig: mess up both again to show that relative distortion remains constant
# new_cloud = lc(new_cloud, -m_hat)
# old_cloud = lc(old_cloud, -m_hat)
# old_cloud = lc(old_cloud, 0.2*m_hat)

# #remove ground plane
# old_cloud = old_cloud[old_cloud[:,2] > -1] 
# new_cloud = new_cloud[new_cloud[:,2] > -1] 

# Rotate + Translate new point cloud
trans = X_gt[:3]
rot = R_tf(X_gt[3:]).numpy()
new_cloud = (new_cloud @ rot) + trans

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(old_cloud, c = "#CB2314")) 
disp.append(Points(new_cloud, c = "#2c7c94")) 
plt.show(disp, "raw point clouds")
ViewInteractiveWidget(plt.window)

### Attempt to solve with basic 6 state solution (Impossible)

In [16]:
from linear_corrector import LC
pc1 = old_cloud
pc2  = new_cloud
m_hat0 = np.array([0, 0, 0, 0, 0, 0.])
dc = LC(cloud1 = pc2, cloud2 = pc1, fid = 50, niter = 20, draw = True, 
        m_hat0 = m_hat0,  mnp = 25, RM = False, solver='6_state')
ViewInteractiveWidget(dc.plt.window)


~~~~~~~~~~~Iteration  0 ~~~~~~~~~~
took 0.01751232147216797 sec  to apply motion profile
took 0.0020051002502441406 sec to get H
m_hat:  [-7.13420749e-01 -1.34362951e-01  1.78663863e-03  3.62022198e-04
 -6.04504079e-04  1.77615490e-02]
~~~~~~~~~~~Iteration  1 ~~~~~~~~~~
took 0.014454126358032227 sec  to apply motion profile
took 0.0020067691802978516 sec to get H
m_hat:  [-8.56980681e-01 -3.90313685e-01  2.20677839e-03  3.69397982e-04
 -6.33577176e-04 -8.96808505e-03]
~~~~~~~~~~~Iteration  2 ~~~~~~~~~~
took 0.016089916229248047 sec  to apply motion profile
took 0.002033710479736328 sec to get H
m_hat:  [-9.57634568e-01 -5.12882054e-01  1.99181889e-03  3.02958593e-04
 -5.67271607e-04 -4.59617153e-02]
~~~~~~~~~~~Iteration  3 ~~~~~~~~~~
took 0.01579570770263672 sec  to apply motion profile
took 0.0020627975463867188 sec to get H
m_hat:  [-9.30355251e-01 -1.08090031e+00  2.23609433e-03  3.32610653e-04
 -6.79788354e-04 -1.93535775e-01]
~~~~~~~~~~~Iteration  4 ~~~~~~~~~~
took 0.0145916938781

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

### Run Newton-Raphson with a priori correspondences

In [130]:
A_hat = np.array([0., 0, 0, 0, 0, 0, 
                  0, 0, 0, 0, 0, 0])
skip = 1 #50
y_i = new_cloud[::skip] #baseline
y_j = old_cloud[::skip] #distorted cloud

print("num corr total:", len(y_i))

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(y_i[:,:3], c = "#a65852 ", alpha = 0.5, r=5.5)) #

runlen = 5
for count in range(runlen):
    
    print("~~~~ iteration ", count, "~~~~~~~~~~~")
    print("A_hat = \n", np.round(A_hat[:6],4), "\n", np.round(A_hat[6:],4)) 
    
    #decompose A_hat into X_hat and m_hat
    X_hat = A_hat[:6] 
    m_hat = A_hat[6:]
#     X_hat = np.array([0, 0, 0, 0, 0, 0.]) #for debug
#     m_hat = np.array([0, 0, 0, 0, 0, 0.]) #for debug
#     m_hat = 0.1*m_hat
    
    #apply last estimate of distortion correction
#     print("m_hat", m_hat)
    y_j_undistort = lc(y_j, m_hat)

    #apply last rigid transform
    rot = R_tf(X_hat[3:]).numpy()
    trans = X_hat[:3]
    y_j_undistort = (y_j_undistort @ rot) + trans
    
#     print("rot: \n", rot,"\n trans: \n", trans)  
#     print("\n y_i \n",np.shape(y_i), "\n", y_i[:3])
#     print("y_j_undistort \n",np.shape(y_j_undistort), "\n", y_j_undistort[:3])

    
    #get H = [H_X, H_m]
    #get jacobain of distortion correction function
    H_m = dc.get_H_m(y_j_undistort, m_hat) 
#     print("\n H_m:", np.shape(H_m), "\n", H_m[:10])
    
    #get jacobian of rigid transform function 
    H_x = jacobian_tf(tf.transpose(tf.convert_to_tensor(y_j_undistort, tf.float32)), tf.convert_to_tensor(X_hat[3:], tf.float32)) # shape = [num of corr * 3, 6]
    #need to append on a row of zeros since we are working with homogeneous coordinates!
    H_x = tf.reshape(H_x, (tf.shape(H_x)[0]//3, 3, 6)) # -> need shape [#corr//4, 4, 6]
#     print("\n H_x before:", np.shape(H_x), "\n", H_x[0])
    H_x = tf.concat([H_x, tf.zeros([len(H_x),1,6])], axis = 1)
    H_x = tf.reshape(H_x, (-1, 6))
#     print("\n H_x after:", np.shape(H_x), "\n", H_x[:10])
    H_x = H_x.numpy()
        
    #delta_A =  ((H^T*H)^-1)(H^T)(yi - yj_undistort)
    residual = (np.append(y_i, np.ones([len(y_i),1]), axis = 1) -
                np.append(y_j_undistort, np.ones([len(y_i),1]), axis = 1)).flatten()
#     print("residual", np.shape(residual), "\n", residual)
    
    H = np.append(H_x, H_m, axis = 1)
    print("H: \n", np.shape(H))

    print("pinv(HTH): \n", np.shape(np.linalg.pinv(H.T @ H)))
    print("pinv(HTH) @ H.T: \n", np.shape(np.linalg.pinv(H.T @ H) @ H.T))
    
    delta_A = np.linalg.pinv(H.T @ H) @ H.T @ residual
    print("\n delta_A \n", np.round(delta_A[:6], 5), "\n", np.round(delta_A[6:], 5))
    #augment rigid transform components
    A_hat[:6] -=   delta_A[:6]
    #augment distortion components
    A_hat[6:9] -= delta_A[6:9]
    A_hat[9:] += delta_A[9:]

    #plot updated cloud2
#     color = [0.5 + count/(runlen*2), 1 - (count+1)/runlen, (count+1)/runlen]
#     disp.append(Points(y_j_undistort[:,:3], c = color, r=3.5))
    disp.append(Points(y_j_undistort[:,:3], c = "#2c7c94 ", alpha = (count+1)/(runlen+1), r=3.5))

    
plt.show(disp, "12 State Solution")
ViewInteractiveWidget(plt.window)

num corr total: 18074
~~~~ iteration  0 ~~~~~~~~~~~
A_hat = 
 [0. 0. 0. 0. 0. 0.] 
 [0. 0. 0. 0. 0. 0.]
H: 
 (72296, 12)
pinv(HTH): 
 (12, 12)
pinv(HTH) @ H.T: 
 (12, 72296)

 delta_A 
 [ 1.4764e+00  6.8890e-02  1.0410e-02 -2.4200e-03 -6.0000e-05 -1.0541e-01] 
 [-2.36395e+00 -3.74720e-01 -8.01000e-03 -4.50000e-03  1.16000e-03
 -1.09400e-02]
~~~~ iteration  1 ~~~~~~~~~~~
A_hat = 
 [-1.4764e+00 -6.8900e-02 -1.0400e-02  2.4000e-03  1.0000e-04  1.0540e-01] 
 [ 2.3639e+00  3.7470e-01  8.0000e-03 -4.5000e-03  1.2000e-03 -1.0900e-02]
H: 
 (72296, 12)
pinv(HTH): 
 (12, 12)
pinv(HTH) @ H.T: 
 (12, 72296)

 delta_A 
 [ 2.547e-02 -8.010e-02 -1.080e-02  2.760e-03  8.000e-05  6.050e-03] 
 [-0.07579  0.38269  0.00615  0.00536 -0.00075  0.01262]
~~~~ iteration  2 ~~~~~~~~~~~
A_hat = 
 [-1.5019e+00  1.1200e-02  4.0000e-04 -3.0000e-04 -0.0000e+00  9.9400e-02] 
 [ 2.4397e+00 -8.0000e-03  1.9000e-03  9.0000e-04  4.0000e-04  1.7000e-03]
H: 
 (72296, 12)
pinv(HTH): 
 (12, 12)
pinv(HTH) @ H.T: 
 (12, 72296)

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

# Run again with voxel-based correspondences

In [194]:
from linear_corrector import LC

dc = LC(cloud1 = new_cloud, cloud2 = old_cloud, fid = 50, niter = 20, 
        draw = True, mnp = 25, RM = False, solver = '12_state', 
        max_buffer = 1.0)
ViewInteractiveWidget(dc.plt.window)

~~~~~~~~~~~Iteration  0 ~~~~~~~~~~
num corr: 
 tf.Tensor([154], shape=(1,), dtype=int32)
H: 
 tf.Tensor([154   3  12], shape=(3,), dtype=int32)
LUT tf.Tensor([154   3   3], shape=(3,), dtype=int32)

 residuals (462, 1)

 delta_A
 (12,)
A: 
 [-1.5274 -0.3084  0.5092 -0.0206  0.1689  0.1518] 
 [ 1.9316  0.7469 -0.9408  0.1191 -0.3197 -0.1948]
~~~~~~~~~~~Iteration  1 ~~~~~~~~~~
num corr: 
 tf.Tensor([232], shape=(1,), dtype=int32)
H: 
 tf.Tensor([232   3  12], shape=(3,), dtype=int32)
LUT tf.Tensor([232   3   3], shape=(3,), dtype=int32)

 residuals (696, 1)

 delta_A
 (12,)
A: 
 [-2.2281 -0.2679  0.5417  0.0289  0.1982  0.1264] 
 [ 2.4777e+00  7.1730e-01 -1.2620e+00 -2.4000e-03 -2.7910e-01 -7.0900e-02]
~~~~~~~~~~~Iteration  2 ~~~~~~~~~~
num corr: 
 tf.Tensor([257], shape=(1,), dtype=int32)
H: 
 tf.Tensor([257   3  12], shape=(3,), dtype=int32)
LUT tf.Tensor([257   3   3], shape=(3,), dtype=int32)

 residuals (771, 1)

 delta_A
 (12,)
A: 
 [-2.1117 -0.1458  0.2581  0.104   0.1141  0.0973]

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)